In [1]:
from tqdm import tqdm
from collections import defaultdict
from copy import deepcopy
import os
from pprint import pprint

from pycocotools.coco import COCO
import numpy as np
from mmengine import dump, load

from utils import box_xywh2xyxy

In [12]:
train_img = load('../../annotations/image_info_train.json')
test_img  = load('../../annotations/image_info_test.json')

# person keypoints

In [10]:
# coco_pose = load('/data0/DataSet/COCO2017/annotations/person_keypoints_val2017.json')
# pose_category = coco_pose['categories']
# dump(pose_category, 'pose_categories.json')

pose_categories = load('../../configs/pose_categories.json')

In [14]:
def gen_pose_pseudo_ann(coco_ins: COCO, pose_props: dict, person_id: int = 1):
    annotations = []
    for img_id, ins_anns in tqdm(coco_ins.imgToAnns.items()):
        filename = coco_ins.loadImgs(img_id)[0]['file_name']
        preds = pose_props[filename]
        pose_preds = preds['person_keypoints']
        human_bbox_preds = preds['instances']['bboxes'][pose_preds['tgt_ins_id']]
        pose_preds = np.concatenate([
            pose_preds['keypoints'], 
            pose_preds['scores'][..., None]
        ], axis=-1)

        for ann in ins_anns:
            if ann['category_id'] != person_id:
                continue

            bbox = np.array(ann['bbox'])
            bbox[2:] += bbox[:2]

            match = (human_bbox_preds == bbox).all(1)
            matched_pred = np.where(match)[0]
            assert matched_pred.shape[0] == 1

            pose_ann = deepcopy(ann)
            pose_ann.update({
                'keypoints': pose_preds[matched_pred],
                'num_keypoints': 17,
            })
            annotations.append(pose_ann)

    coco_pose = deepcopy(coco_ins.dataset)
    coco_pose['annotations'] = annotations
    coco_pose['categories' ] = pose_categories
    return coco_pose

## 由VIPLO提供

In [14]:
proposal_root = '../../proposals/ViPLO/'
test_total_prop_path = os.path.join(proposal_root, 'proposal_viplo_test.pkl')
test_pose_pseudo_path = '../../annotations/person_keypoints_test_viplo.json'

In [15]:
coco_ins_test = COCO('../../annotations/instances_test.json')

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


In [16]:
test_proposals = load(test_total_prop_path)
len(test_proposals)

9546

In [20]:
# person_keypoints
test_key_ann = gen_pose_pseudo_ann(coco_ins_test, test_proposals)
dump(test_key_ann, test_pose_pseudo_path)

100%|██████████| 9546/9546 [00:00<00:00, 9952.30it/s] 


## 由 mmpose-vitpose-l 提供

In [2]:
proposal_root = '../../proposals/GT/'
train_total_prop_path = os.path.join(proposal_root, 'proposal_gt_train.pkl')
test_total_prop_path  = os.path.join(proposal_root, 'proposal_gt_test.pkl')
train_pose_pseudo_path = '../../annotations/person_keypoints_train_vitpose.json'
test_pose_pseudo_path  = '../../annotations/person_keypoints_test_vitpose.json'

In [3]:
train_proposals = load(train_total_prop_path)
test_proposals  = load( test_total_prop_path)
len(train_proposals), len(test_proposals)

(37633, 9546)

In [4]:
coco_ins_train = COCO('../../annotations/instances_train.json')
coco_ins_test  = COCO('../../annotations/instances_test.json')

loading annotations into memory...
Done (t=1.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!


In [15]:
# person_keypoints
train_key_ann = gen_pose_pseudo_ann(coco_ins_train, train_proposals)
test_key_ann  = gen_pose_pseudo_ann(coco_ins_test ,  test_proposals)
dump(train_key_ann, train_pose_pseudo_path)
dump( test_key_ann,  test_pose_pseudo_path)

100%|██████████| 9546/9546 [00:01<00:00, 8459.30it/s] 


# Instance

In [ ]:
def cvt_valid_pred2anns(preds, tgt):
    ins_anns = []
    person_anns = []
    valid_preds = []
    valid_persons = []
    score_threshold = 0.3

    ins_id = 0
    person_id = 0
    for pred in preds:
        score = pred['score']
        if score < score_threshold:
            continue
        bbox = pred['bbox']
        bbox[0] = max(0, int(bbox[0]))
        bbox[1] = max(0, int(bbox[1]))
        bbox[2] = max(1, int(bbox[2]))
        bbox[3] = max(1, int(bbox[3]))
        area = bbox[2] * bbox[3]
        if area < 10:
            continue
        pred['bbox'] = bbox
        ins_ann = {
            'bbox': bbox,
            'category_id': pred['category_id'],
            'image_id': pred['image_id'] + 1,  # FIXME 
            'iscrowd': 1,  # NOTE 
            'id': ins_id,
            'score': score,
            'area': area,
            'segmentation': pred['segmentation']  # NOTE RLE format
        }
        ins_anns.append(ins_ann)
        ins_id += 1
        valid_preds.append(pred)
        if pred['category_id'] == 1:
            person_ann = ins_ann
            person_ann['id'] = person_id
            person_anns.append(person_ann)
            valid_persons.append(pred)
    ins_anns = {
        'images': tgt['images'], 
        'annotations': ins_anns, 
        'categories': tgt['categories']
    }
    return ins_anns, valid_preds, valid_persons

In [ ]:
output_train_root = '../../predictions/instance/train/MaskDINO'
output_test_root = '../../predictions/instance/test/MaskDINO'

In [ ]:
# trainset
preds_train = load(os.path.join(output_train_root, "coco_instances_results.json"))
len(preds_train)
tgt_train = load('../../annotations/train_hoi2instances.json')
new_anns_train, kept_boxes_train, persons_train = cvt_valid_pred2anns(preds_train, tgt_train)
dump(new_anns_train  , os.path.join(output_train_root, 'instance_valid2ann.json'))
dump(kept_boxes_train, os.path.join(output_train_root, 'instance_valid.json'    ))
dump(persons_train   , os.path.join(output_train_root, 'person_valid.json'      ))

In [ ]:
# testset
preds_test = load(os.path.join(output_test_root, "coco_instances_results.json"))
len(preds_test)
tgt_test = load('../../annotations/test_hoi2instances.json')
new_anns_test, kept_boxes_test, persons_test = cvt_valid_pred2anns(preds_test, tgt_test)
dump(new_anns_test  , os.path.join(output_test_root, 'instance_valid2ann.json'))
dump(kept_boxes_test, os.path.join(output_test_root, 'instance_valid.json'    ))
dump(persons_test   , os.path.join(output_test_root, 'person_valid.json'      ))